# DNA Replication Initiation In Syn3A

In this tutorial, you will learn:

1) How to restart CME simulation to update the rate constants due to the volume change in the whole cell cycle

2) How to export trajectories in LM file to CSV file

3) 

The replication initiation process in Syn3A is modeled by the binding of DnaA with oriC, the very specific region of chromosome near gene JCVISYN3A_0001.

The initiation can be divided into three stages in our simulation. The first stage is the binding of DnaA with high and low affinity signatures to open the ssDNA pocket. The second stage is the binding of DnaA with AT rich ssDNA region to grow a DnaA filament and the threshold of the number of DnaA on ssDNA is 30, above which the initiation completes. The last is the fast removal of DnaA from the ssDNA when the replisome (whose assembly is not modeled) starts to move along the chromosome.

References:

Z. Thornburg, Marcelo Melo, David Bianchi, Troy Brier, Cole Crotty, Marian Breuer, Hamilton Smith, Clyde Hutchison, John Glass, and Zaida Luthey-Schulten, Modelling the genetic information processes in a minimal cell, Frontiers in Molecular Biosciences, 2019 https://doi.org/10.3389/fmolb.2019.00130

Zane R. Thornburg, David M. Bianchi, Troy A. Brier, Benjamin R. Gilbert, Tyler M. Earnest, Marcelo C.R. Melo, Nataliya Safronova, James P. Sáenz, András T. Cook, Kim S. Wise, Clyde A. Hutchison III, Hamilton O. Smith, John I. Glass, Zaida Luthey-Schulten, Fundamental behaviors emerge from simulations of a living minimal cell, Cell, 2022, https://doi.org/10.1016/j.cell.2021.12.0

In [1]:
# pyLM to construct the CME model
from pyLM import *
from pyLM.units import *

import math as m
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import csv
import pandas as pd
import importlib
import h5py
import sys

import func_initiation as f

## Run Simulation

Three-layer loop to perform the simulation of multiple replicates' whole cell cycle with/without volume change where the CME simulation restarts.

The first layer to iterate with or without volume change; The second layer to iterate through all the replicates; The third layer to iterate the whole cell cycle by restarting CME simuation

Each cell cycle's trajectory is written to an individual csv file.

In [ ]:
log = open('./initiation_CME.log', 'w')

original_stdout = sys.stdout
original_stderr = sys.stderr

sys.stdout = log
sys.stderr = log

#In each simulation of whole cell cycle, restart the simulation per 60 seconds to update the cell volume
restartInterval = 60 # s

# Simulation reps replicates serially
reps = 5

# Initial value of cell radius is 200 nm
r_cell = 2.0*(10**-7) # m
cellVolume_init = (4*np.pi/3)*1000*r_cell**3 # L

# Time needed to double the cell volume is 60 minutes
doublingTime = 60 # min

# The length of whole cell cycle is 105 minutes.
endtime = 105  # minutes

# Three-layer loop
csvfilefolers = ['./WithVolumeChange/', './WithoutVolumeChange/']
volumeChange_flag = [True, False]

for i_change, change_flag in enumerate(volumeChange_flag):

    csvfilefolder = csvfilefolers[i_change]

    if not os.path.exists(csvfilefolder):
        os.makedirs(csvfilefolder)

    # Get the species list and initial counts
    species, ini_counts = f.speciesandIniCounts()

    for rep in range(1,reps+1):
        print('Start the simulation of '+str(rep) +' replicate')
        
        counts = ini_counts
        
        trajDF = pd.DataFrame()
        
        for time in range(0,endtime):

            # Filename of the current simulation
            filename = 'Ini_' + str(time+1) + '.lm'

            # Linear increase to double volume then contains
            cellVolume = f.volumeChange(time, doublingTime, cellVolume_init)
            
            # Initialize each minute's simulation
            sim=CME.CMESimulation(name="DNA Replication Initiation in Syn3A" + str(time+1))
            # Write the trajectories of each species per 1 second
            sim.setWriteInterval(1)
            sim.setSimulationTime(restartInterval)
            
            # define the reactions and particles numbers every simulation
            f.addRepInit(sim, cellVolume, counts, species, change_flag)

            sim.save(filename)
            sim.run(filename, "lm::cme::GillespieDSolver", 1)
            
            # counts is the number of each species at last frame written time
            # trajDF is the record of numbers of all frame written time in all simulation
            counts,trajDF = f.recordTraces(trajDF, filename, time, restartInterval, species)


            # Remove lm file from last whole cell cycle simulation
            os.system("rm -rf %s"%(filename))
        print('Finish the simulation of '+str(rep) +' replicate')
        
        # Write the trajectory to csv file
        csvpath = '{0}initiation_{1}.csv'.format(csvfilefolder, rep)
        trajDF.to_csv(csvpath, index = False)


sys.stdout = original_stdout
sys.stderr = original_stderr 